In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import io
from google.colab import drive
import tensorflow as tf

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Data.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
def cleaninTxt(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    
    
    text = text.lower()
    return text

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

df['sentiment']=le.fit_transform(df['sentiment'])
df['review']=df['review'].apply(cleaninTxt)
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["review"].values, df["sentiment"].values, test_size=0.2) 


In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 

In [ ]:
encoded_dict = tokenizer.encode_plus(
                          X_train[0],  
                          truncation = True,                    
                          add_special_tokens = True, 
                          max_length = 256,           
                          padding = 'max_length',
                          return_attention_mask = True,   
                          return_token_type_ids = True     
                    )
encoded_dict

{'input_ids': [101, 1996, 8068, 2003, 2019, 17077, 2092, 2589, 1998, 2200, 4086, 2460, 1012, 2296, 5783, 1997, 1996, 3538, 8456, 1999, 12005, 26243, 2075, 1037, 5760, 6832, 3433, 2000, 1996, 5896, 1012, 2092, 6051, 1010, 2856, 1010, 2915, 1998, 2517, 1012, 1045, 2001, 4527, 2000, 2963, 2008, 2045, 2001, 2053, 17887, 2077, 5008, 1010, 2025, 2130, 1037, 3191, 2083, 1012, 1996, 4616, 3233, 2004, 9025, 2000, 2070, 2986, 12753, 8787, 3772, 1999, 3433, 2000, 1037, 2092, 2517, 5896, 1012, 1996, 19910, 2147, 2001, 6581, 1998, 2045, 2001, 2196, 2019, 12407, 2008, 2037, 2147, 2052, 7540, 2046, 1996, 11850, 23069, 1012, 2625, 2003, 2172, 2488, 1999, 2023, 2553, 1012, 2023, 2143, 2003, 1037, 3539, 2742, 1997, 2129, 2122, 2659, 5166, 8311, 5770, 5889, 2004, 2092, 2004, 2143, 11153, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def l2encode(texts):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for text in texts:

      encoded_dict = tokenizer.encode_plus(
                          text,  
                          truncation = True,                    
                          add_special_tokens = True, 
                          max_length = 256,           
                          padding = 'max_length',
                          return_attention_mask = True,   
                          return_token_type_ids = True     
                    )
      
      input_ids.append(encoded_dict['input_ids'])
      
      attention_masks.append(encoded_dict['attention_mask'])

      token_type_ids.append(encoded_dict['token_type_ids'])

  return input_ids, attention_masks, token_type_ids

def inputs_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

In [ ]:
input_ids_train, attmasks_train , token_types_train = l2encode(X_train)
train_dataset = tf.data.Dataset.from_tensor_slices((input_ids_train, attmasks_train, token_types_train, y_train)).map(inputs_to_dict).shuffle(10000).batch(16)
input_ids_test, attmasks_test , token_types_test = l2encode(X_test)
test_dataset = tf.data.Dataset.from_tensor_slices((input_ids_test, attmasks_test, token_types_test, y_test)).map(inputs_to_dict).batch(16)

In [ ]:
from transformers import TFBertForSequenceClassification
learning_rate = 2e-5
number_of_epochs = 2
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                        num_labels = 2,
                                                        output_attentions = False,
                                                        output_hidden_states = False
                                                        )

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss=model.compute_loss ,optimizer=optimizer, 
                           metrics=['accuracy'])
print(model.summary())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(train_dataset, validation_data=test_dataset, epochs=number_of_epochs)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr